In [8]:
import os
from pathlib import Path

# Rutas YA listas (sin .zip)
DATA_ROOT = "/kaggle/input/data-final/Data"
DATA_YAML = os.path.join(DATA_ROOT, "data.yaml")
YOLO_ROOT = "/kaggle/working/Yolo_Experiments_2"  # ← AQUÍ DONDE SE GUARDAN LOS RESULTADOS

# Crear carpeta de experimentos
os.makedirs(YOLO_ROOT, exist_ok=True)

print("📁 Dataset montado en:", DATA_ROOT)
print("✅ data.yaml existe:", os.path.exists(DATA_YAML))
print("📂 YOLO_ROOT creado en:", YOLO_ROOT)

# Verificar estructura
print("\n📂 Contenido dataset:")
for item in os.listdir(DATA_ROOT):
    item_path = os.path.join(DATA_ROOT, item)
    if os.path.isdir(item_path):
        img_count = len(os.listdir(os.path.join(item_path, "images")))
        print(f"  📁 {item}/ ({img_count} imágenes)")
    else:
        print(f"  📄 {item}")


📁 Dataset montado en: /kaggle/input/data-final/Data
✅ data.yaml existe: True
📂 YOLO_ROOT creado en: /kaggle/working/Yolo_Experiments_2

📂 Contenido dataset:
  📄 data.yaml
  📁 valid/ (150 imágenes)
  📁 test/ (134 imágenes)
  📁 train/ (7771 imágenes)


In [1]:
!pip install ultralytics optuna -q

In [2]:
from ultralytics import YOLO
import torch
import os
import optuna
import json

In [3]:
DEVICE = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print("Device:", DEVICE)


Device: cuda


In [10]:
# Ver qué dice tu data.yaml actual
with open("/kaggle/input/data-final/Data/data.yaml", "r") as f:
    content = f.read()
    print("CONTENIDO ACTUAL:")
    print(content)


CONTENIDO ACTUAL:
path: .

train: train/images
val: valid/images
test: test/images

nc: 4
names: ['door', 'window', 'balcony', 'railing']



In [11]:
# Crear data.yaml con path ABSOLUTO + rutas relativas
DATA_ROOT = "/kaggle/input/data-final/Data"
DATA_YAML_FIXED = "/kaggle/working/data.yaml"

data_yaml_content = f"""path: {DATA_ROOT}
train: train/images
val: valid/images
test: test/images

nc: 4
names: ['door', 'window', 'balcony', 'railing']
"""

with open(DATA_YAML_FIXED, 'w') as f:
    f.write(data_yaml_content)

print("✅ data.yaml CORREGIDO:")
print(data_yaml_content)
print(f"\n📁 Guardado en: {DATA_YAML_FIXED}")


✅ data.yaml CORREGIDO:
path: /kaggle/input/data-final/Data
train: train/images
val: valid/images
test: test/images

nc: 4
names: ['door', 'window', 'balcony', 'railing']


📁 Guardado en: /kaggle/working/data.yaml


In [12]:
DATA_YAML = "/kaggle/working/data.yaml" 

def objective(trial):
    # Hyperparametros para optuna
    lr = trial.suggest_float("lr", 1e-4, 1e-2, log=True)
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "AdamW"])

    # Modelo
    model = YOLO("yolov8l.pt")

    model.train(
        data=DATA_YAML,
        epochs=10, 
        patience=3,  # para si no mejora mAP durante 3 épocas consecutivas
        imgsz=512,
        batch=-1,  # YOLOv8 calcula AUTOMÁTICAMENTE el máximo batch_size que cabe en tu GPU
        lr0=lr,
        optimizer=optimizer_name,
        freeze=10,      # congelar primeras capas (backbone)
        device=DEVICE,  # cuda
        project=YOLO_ROOT,
        name=f"trial_{trial.number}",
        save=True,      # guardar best.pt / last.pt
        exist_ok=True,  # no sobrescribir conflictos
        cache=False,
        verbose=True
    )

    # Evaluación final
    val_results = model.val()

    return 1 - float(val_results.box.map50)


In [13]:
# Crear estudio 
study = optuna.create_study(direction="minimize")

# Ejecutar Optuna
study.optimize(objective, n_trials=3)

print("Mejor configuración encontrada:")
print(study.best_params)


[I 2026-01-24 04:19:54,463] A new study created in memory with name: no-name-38db5917-6921-401e-b8b8-37369b69ae38


engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=-1, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=10, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=512, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.009311958515851737, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8l.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=trial_0, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mask=True, patience=3, perspective=0.0, plots=True, pose=12.0, pretrained=True, profile=False, project=/kaggle/working/Yo

[I 2026-01-24 04:49:43,684] Trial 0 finished with value: 0.26701214559036124 and parameters: {'lr': 0.009311958515851737, 'optimizer': 'AdamW'}. Best is trial 0 with value: 0.26701214559036124.


engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=-1, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=10, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=512, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0025374738430599376, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8l.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=trial_1, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mask=True, patience=3, perspective=0.0, plots=True, pose=12.0, pretrained=True, profile=False, project=/kaggle/working/Y

[I 2026-01-24 05:20:12,342] Trial 1 finished with value: 0.2283620695253017 and parameters: {'lr': 0.0025374738430599376, 'optimizer': 'AdamW'}. Best is trial 1 with value: 0.2283620695253017.


engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=-1, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=10, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=512, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.005133073345812616, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8l.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=trial_2, nbs=64, nms=False, opset=None, optimize=False, optimizer=Adam, overlap_mask=True, patience=3, perspective=0.0, plots=True, pose=12.0, pretrained=True, profile=False, project=/kaggle/working/Yol

[I 2026-01-24 05:51:02,243] Trial 2 finished with value: 0.25103311092511693 and parameters: {'lr': 0.005133073345812616, 'optimizer': 'Adam'}. Best is trial 1 with value: 0.2283620695253017.


Mejor configuración encontrada:
{'lr': 0.0025374738430599376, 'optimizer': 'AdamW'}


In [18]:
# RESUMEN FINAL Optuna
print("🏆 RESULTADOS FINALES Optuna:")
print(f"Total trials: {len(study.trials)}")
print(f"Mejor trial: {study.best_trial.number}")
print(f"Mejor params: {study.best_params}")
print(f"Mejor mAP50: {1 - study.best_value:.3f}")

🏆 RESULTADOS FINALES Optuna:
Total trials: 3
Mejor trial: 1
Mejor params: {'lr': 0.0025374738430599376, 'optimizer': 'AdamW'}
Mejor mAP50: 0.772


In [ ]:
# Cargar el mejor trial
best_trial_number = study.best_trial.number
best_trial_dir = os.path.join(YOLO_ROOT, f"trial_{best_trial_number}")
best_model_path = os.path.join(best_trial_dir, "weights", "best.pt")

# Cargar modelo YOLO
from ultralytics import YOLO
best_model = YOLO(best_model_path)

# Evaluar validación final
results = best_model.val(data=DATA_YAML)

print(f"\n VALIDACIÓN:")
print(f"mAP50:     {results.box.map50:.3f}")
print(f"mAP50-95:  {results.box.map:.3f}")
print(f"Precision: {results.box.mp:.3f}")
print(f"Recall:    {results.box.mr:.3f}")

Ultralytics 8.4.7 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 113 layers, 43,609,692 parameters, 0 gradients, 164.8 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 24.2±19.4 MB/s, size: 39.3 KB)
val: Scanning /kaggle/input/data-final/Data/valid/labels... 150 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 150/150 590.3it/s 0.3s0.1s
val: /kaggle/input/data-final/Data/valid/images/Fv1_0950.jpg: 1 duplicate labels removed
val: /kaggle/input/data-final/Data/valid/images/Fv1_0952.jpg: 1 duplicate labels removed
val: /kaggle/input/data-final/Data/valid/images/Fv1_0957.jpg: 1 duplicate labels removed
val: /kaggle/input/data-final/Data/valid/images/Fv1_0961.jpg: 1 duplicate labels removed
val: /kaggle/input/data-final/Data/valid/images/Fv1_0968.jpg: 1 duplicate labels removed
val: /kaggle/input/data-final/Data/valid/images/Fv1_0977.jpg: 1 duplicate labels removed
val: /kaggle/input/data-final/Data/valid/images/Fv1_0982.jpg: 3 duplicate lab

In [ ]:
# Cargar MEJOR config encontrada
best_params = study.best_params
model = YOLO("yolov8l.pt")  

results = model.train(
    data=DATA_YAML,
    epochs=25,                    # ← MÁS épocas
    lr0=best_params['lr'],        # ← MEJOR lr
    optimizer=best_params['optimizer'],  # ← MEJOR optimizer
    patience=3,                   # Early stopping
    project=YOLO_ROOT,
    name="FINAL_MEJORADO"
)


Ultralytics 8.4.7 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=25, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.0025374738430599376, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8l.pt, momentum=0.937, mosaic=1.0, multi_scale=0.0, name=FINAL_MEJORADO, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mask=True, patience=3, perspe

In [ ]:
# 1. CARGAR MODELO FINAL
model = YOLO(f"{YOLO_ROOT}/FINAL_MEJORADO/weights/best.pt")

# 2. VALIDACIÓN (val dataset) 
print("VALIDACIÓN:")
val_results = model.val(data=DATA_YAML, split='val')
print(f"Val mAP50: {val_results.box.map50:.3f}")


In [ ]:
# 3. **TEST FINAL** (test dataset) ← ¡TU NÚMERO OFICIAL!
print("\nTEST FINAL:")
test_results = model.val(data=DATA_YAML, split='test')
print(f"**TEST mAP50:** {test_results.box.map50:.3f}")
print(f"Test mAP50-95: {test_results.box.map:.3f}")
print(f"Test Precision: {test_results.box.mp:.3f}")
print(f"Test Recall: {test_results.box.mr:.3f}")

In [ ]:
# 4. FINALIZAR
!cp "{YOLO_ROOT}/FINAL_MEJORADO/weights/best.pt" /kaggle/working/FINAL_TESTED.pt
print("\n /kaggle/working/FINAL_TESTED.pt → Download")